# English Correction with Langchain

# Overview

## Step 0: Configuring the Environment 

In [ ]:
# Required Libraries
!pip install -r ../requirements.txt --quiet

In [ ]:
# System
import os
import sys

# Add src directory to system path
sys.path.append(os.path.abspath('../src'))

# Internal Modules
from main import GitHubMarkdownProcessor
from utils import load_config_and_secrets


### Define Constants and Paths

In [ ]:
CONFIG_PATH = "../configs/configs.yaml"
SECRETS_PATH = "../configs/secrets.yaml"

### Configuration and Secrets Loading

In [ ]:
config, secrets = load_config_and_secrets(CONFIG_PATH, SECRETS_PATH)

## Step 1: Extracting and Parsing Markdown Files From GitHub Repositories

In [ ]:
# Repo URL and token
repo_url = "https://github.com/HPInc/AI-Blueprints"
access_token = secrets.get("GITHUB_ACCESS_TOKEN")

# Create processor instance
processor = GitHubMarkdownProcessor(repo_url=repo_url, access_token=access_token)

# Run preprocessing workflow
processor.run()